In [2]:
import os, sys
import numpy as np
import qcportal as ptl
import pandas as pd


In [13]:
def load_QM9(input_file=None, output_file=None):
    """Load the QM9 datset sourced from qcportal
    
    Parameters
    ----------
    input_file : str, optional, default=None
        The locally stored hdf5 file of the QM9 dataset to load. 
        If not provided, the data will be fetched from qcarchive.
    output_file : str, optional, default=None
        The name defining where to save the dataset locally.
        If input_file and output_file are both None, the dataset will be deleted upon exit.

    Returns
    --------
    qcportal.collections.dataset.Dataset, 
        The qm9 dataset
    """
    qcp_client = ptl.FractalClient()

    # to get QM9 from qcportal, we need to define which collection and QM9
    # we will first check to see if it exists 
    qcportal_data = {'collection': 'Dataset', 'dataset': 'QM9'}

    try: 
        qm9 = qcp_client.get_collection(qcportal_data['collection'], qcportal_data['dataset'])
    except:
        print(f"Dataset {qcportal_data['dataset']} is not available in collection {qcportal_data['collection']}.")
    
    if input_file is None:
        # if we don't define output_file below, the data we download
        # will be deleted upon exit.
        qm9.download()

        if not output_file is None:
            #need some error checking here.  probably want to raise errors in the real code
            assert output_file.endswith('.hdf5')
            qm9.to_file(path=output_file, encoding='hdf5')
            
    else:
        # again, better error checking is probably required
        # but this is probably good enough
        assert input_file.endswith('.hdf5')
        qm9.set_view(input_file)

    
    return qm9
    

In [24]:
def parse_data(qm9):
    
    """Parse the QM9 datset into a dictionaries

    Note: this function takes a while to run, as retrieving the records
    appears to be a bit slow given the dataset size.
    
    Parameters
    ----------
    qm9 : qcportal.collections.dataset.Dataset
        The Dataset to parse
   
    Returns
    --------
    tuple, (list, dict, dict) 
        The first index is a list of molecule names
        The second index is a dictionary of molecule geometry information, keyed by name.
        the third index is a dictionary of molecule QM properties, keyed by name.
    """
    
    molecules = qm9.get_molecules()

    # just going to restrict to this for demo purposes now
    # this could easily be an argument
    records  = qm9.get_records(method='b3lyp')

    # the order of molecules and records in the dataframes provided by qcportal 
    # is not guaranteed. Use dicts keyed by the named for each property.
    
    records_dict = {} 
    molecules_dict = {}

    names = []
    
    for i in range(molecules.shape[0]):
        name = molecules.iloc[i].name
        molecules_dict[name] = molecules.iloc[i][0]
        names.append(name)
        
    for i in range(len(records)):
        rec = records.iloc[i].record
        name = records.index[i]
        records_dict[name] = rec
        
    return (names, molecules_dict, records_dict)

In [19]:
qm9 = load_QM9(input_file='/Users/cri/Documents/Projects-msk/qcarchive/qm9.hdf5')

In [25]:
names, molecules, records = parse_data(qm9)

Let us just look at the arbitrary molecule from the names list to explore the structure of the information from qcportal. 

Just call an individual molecule will default to visualizing it. 

In [39]:
molecules[names[200]]

NGLWidget()

In [40]:
print(molecules[names[200]])

Molecule(name='C6H8N2', formula='C6H8N2', hash='627bb33')


For each molecule, we can see the info stored via the `dict` function which, as the function name suggests, returns a dictionary of the molecule properties/info.

In [41]:
molecules[names[200]].dict()

{'schema_name': 'qcschema_molecule',
 'schema_version': 2,
 'validated': True,
 'symbols': array(['C', 'C', 'N', 'C', 'C', 'N', 'C', 'C', 'H', 'H', 'H', 'H', 'H',
        'H', 'H', 'H'], dtype='<U1'),
 'geometry': array([[-1.39051210e-01,  2.83318818e+00, -1.65827600e-02],
        [-5.01442800e-02, -6.25411000e-03,  5.31336900e-02],
        [ 3.64649400e-02, -1.19749836e+00, -2.18241027e+00],
        [ 1.16266390e-01, -3.72038431e+00, -2.16145846e+00],
        [ 2.13221080e-01, -5.05322009e+00, -4.66981902e+00],
        [ 1.17849420e-01, -5.18739469e+00, -9.15737600e-02],
        [ 3.14795800e-02, -3.98373004e+00,  2.11532670e+00],
        [-5.59533500e-02, -1.36624272e+00,  2.30630329e+00],
        [ 1.52032710e+00,  3.57197760e+00, -1.00609064e+00],
        [-1.79899626e+00,  3.46631919e+00, -1.07595343e+00],
        [-2.04921830e-01,  3.64663564e+00,  1.87950305e+00],
        [ 1.91393720e+00, -6.22452692e+00, -4.79036663e+00],
        [-1.40453398e+00, -6.32768411e+00, -4.86377329e

We can easily access any of the quantities listed in the dict, e.g., geometry.  Some of the properties have multiple access points, e.g, could also called `molecules[names[200]].geometry`

In [42]:
molecules[names[200]].dict()['geometry']

array([[-1.39051210e-01,  2.83318818e+00, -1.65827600e-02],
       [-5.01442800e-02, -6.25411000e-03,  5.31336900e-02],
       [ 3.64649400e-02, -1.19749836e+00, -2.18241027e+00],
       [ 1.16266390e-01, -3.72038431e+00, -2.16145846e+00],
       [ 2.13221080e-01, -5.05322009e+00, -4.66981902e+00],
       [ 1.17849420e-01, -5.18739469e+00, -9.15737600e-02],
       [ 3.14795800e-02, -3.98373004e+00,  2.11532670e+00],
       [-5.59533500e-02, -1.36624272e+00,  2.30630329e+00],
       [ 1.52032710e+00,  3.57197760e+00, -1.00609064e+00],
       [-1.79899626e+00,  3.46631919e+00, -1.07595343e+00],
       [-2.04921830e-01,  3.64663564e+00,  1.87950305e+00],
       [ 1.91393720e+00, -6.22452692e+00, -4.79036663e+00],
       [-1.40453398e+00, -6.32768411e+00, -4.86377329e+00],
       [ 2.04735130e-01, -3.68692218e+00, -6.21129440e+00],
       [ 3.22407800e-02, -5.16167892e+00,  3.80168817e+00],
       [-1.25360320e-01, -4.29621550e-01,  4.12596607e+00]])

Similarly, we can see the QM related info stored for each record.  

In [43]:
records[names[200]].dict()

{'id': '4897243',
 'hash_index': None,
 'procedure': 'single',
 'program': 'psi4',
 'version': 1,
 'protocols': {},
 'extras': {'qcvars': {'CURRENT DIPOLE X': -0.06979275427846111,
   'CURRENT DIPOLE Y': 1.2963495088345212,
   'CURRENT DIPOLE Z': 1.2935375170372443,
   'CURRENT ENERGY': -342.71692437785396,
   'CURRENT REFERENCE ENERGY': -342.71692437785396,
   'DFT FUNCTIONAL TOTAL ENERGY': -342.71692437785396,
   'DFT TOTAL ENERGY': -342.71692437785396,
   'DFT VV10 ENERGY': 0.0,
   'DFT XC ENERGY': -39.91391192383341,
   'NUCLEAR REPULSION ENERGY': 349.80104044662846,
   'ONE-ELECTRON ENERGY': -1154.8739622133487,
   'PCM POLARIZATION ENERGY': 0.0,
   'SCF DIPOLE X': -0.06979275427846111,
   'SCF DIPOLE Y': 1.2963495088345212,
   'SCF DIPOLE Z': 1.2935375170372443,
   'SCF ITERATION ENERGY': -342.71692437785396,
   'SCF ITERATIONS': 13.0,
   'SCF TOTAL ENERGY': -342.71692437785396,
   'TWO-ELECTRON ENERGY': 502.2699093126997,
   'XC GRID RADIAL POINTS': 100.0,
   'XC GRID SPHERICAL 

The qcvars entry under extras houses a summary of the key QM data. Like geometry, extras can also be accessed either through the dict or its own entry point to get the qcvars info. Important note, if you check the units of the dataset `qm9.units`, it will report kcal/mol, however, the energy values given here are hatree.

In [44]:
records[names[200]].extras['qcvars']

{'CURRENT DIPOLE X': -0.06979275427846111,
 'CURRENT DIPOLE Y': 1.2963495088345212,
 'CURRENT DIPOLE Z': 1.2935375170372443,
 'CURRENT ENERGY': -342.71692437785396,
 'CURRENT REFERENCE ENERGY': -342.71692437785396,
 'DFT FUNCTIONAL TOTAL ENERGY': -342.71692437785396,
 'DFT TOTAL ENERGY': -342.71692437785396,
 'DFT VV10 ENERGY': 0.0,
 'DFT XC ENERGY': -39.91391192383341,
 'NUCLEAR REPULSION ENERGY': 349.80104044662846,
 'ONE-ELECTRON ENERGY': -1154.8739622133487,
 'PCM POLARIZATION ENERGY': 0.0,
 'SCF DIPOLE X': -0.06979275427846111,
 'SCF DIPOLE Y': 1.2963495088345212,
 'SCF DIPOLE Z': 1.2935375170372443,
 'SCF ITERATION ENERGY': -342.71692437785396,
 'SCF ITERATIONS': 13.0,
 'SCF TOTAL ENERGY': -342.71692437785396,
 'TWO-ELECTRON ENERGY': 502.2699093126997,
 'XC GRID RADIAL POINTS': 100.0,
 'XC GRID SPHERICAL POINTS': 302.0,
 'XC GRID TOTAL POINTS': 448207.0}

Also interesting, the entire output from the run can be accessed. 

In [45]:
print(records[names[200]].get_stdout())


  Memory set to  40.533 GiB by Python driver.

*** tstart() called on dt016
*** at Wed Aug 28 19:56:24 2019

   => Loading Basis Set <=

    Name: DEF2-SVP
    Role: ORBITAL
    Keyword: BASIS
    atoms 1-2, 4-5, 7-8 entry C          line    90 file /home/mwelborn/miniconda3/envs/qca/share/psi4/basis/def2-svp.gbs 
    atoms 3, 6          entry N          line   110 file /home/mwelborn/miniconda3/envs/qca/share/psi4/basis/def2-svp.gbs 
    atoms 9-16          entry H          line    15 file /home/mwelborn/miniconda3/envs/qca/share/psi4/basis/def2-svp.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RKS Reference
                        4 Threads,  41506 MiB Core
         ---------------------------------------------------------

  ==> Geometry <==

    Molecular point group: c1
    Fu